In [1]:
package main

import (
    "fmt"
    "math"
    "math/rand"
    "math/cmplx"
    "runtime"
    "time"
)

### packages, variables and functions

In [2]:
func main() {
    rand.Seed(30)
    // random values are deterministic, need to manually change to seed
    fmt.Println(rand.Intn(10))
}

// can be factored like imports
var (
    ToBe   bool       = false
    MaxInt uint64     = 1<<64 - 1
    z      complex128 = cmplx.Sqrt(-5 + 12i)
)
fmt.Printf("%v %v %v\n", ToBe, MaxInt, z)

// omit type when the arguments share the same type
func add(x, y int) int {
    return x + y
}

// can return multiple values
func swap(x string, y string) (string, string) {
    return y, x
}

// named return values: naked return
func split(sum int) (x, y, k int) {
    // will not return the method parameters
    var k = sum / 2
    x = sum * 4 / 9
    y = sum - x
    return
}


main()
add(1, 2)
x, y := swap('a', 'b')
fmt.Print(x)
split(11)

x := uint(3.0)
fmt.Print(x)

false 18446744073709551615 (2+3i)
8
b3

1 <nil>

### flow control statements

In [3]:
// for is the only looping contruct
sum := 0
for i := 0; i < 10; i++ {
    sum += i
}
fmt.Println(sum)

var i int
for i < 10 {
    if i % 2 == 0 {
        i += 1
    } else {
        i += 2
    }
}
i

// variables initialized in the if contruct can be accessible in the else blocks
if v := math.Pow(10, 2); v < 100 {
    fmt.Println("hi")
} else {
    fmt.Println(v)
}

if math.Pow(1, 2) < 100 {
    fmt.Println("yo")
}

// break is automatically applied after each case
switch os := runtime.GOOS; os {
    case "darwin":
        fmt.Println("OS X.")
    case "linux":
        fmt.Println("Linux.")
    default:
        fmt.Printf("%s.\n", os)
    }

switch today := time.Now().Weekday() - 1; today {
case time.Saturday + 0:
    fmt.Println("Sat")
case time.Saturday - 1:
    fmt.Println("Sun")
default:
    fmt.Println("None")
}

45
100
yo
OS X.
Sun


In [4]:
// deferred functions are pushed on to a stack and are executed in LIFO order
func deferfunc() {
    fmt.Println("counting")

    for i := 0; i < 4; i++ {
        defer fmt.Println(i)
    }

    fmt.Println("done")
}
deferfunc()

counting
done
3
2
1
0


### advanced types

#### pointers

In [5]:
z := "hello"
// holds the pointer to var z
p := &z
// denotes the pointers underlying value
*(p)

hello

In [6]:
p

0xc000910270

In [7]:
x := "hello"
&x

0xc0009105e0

In [8]:
x = "helloo"
&x

0xc0009105e0

#### structs

In [9]:
type Vertex struct {
    X int
    Y int
}

func main() {
    v := Vertex{Y:1, X:2}
    p := &v
    
    p.X = 2
    // (*p).Y = 5
    
    fmt.Println(v)
}

main()

{2 1}


#### array & slices

In [10]:
primes := [6]int{2, 3, 5, 7, 11, 13}
// this is a slice, it does not store any data
var s []int = primes[1:4]

// it creates an array and get back the refer as a slice
primes := []int{2, 3, 5, 7, 11, 13}
fmt.Println(s)

s := []struct {
        i int
        b bool
    }{
        {2, true},
        {3, false},
        {5, true},
        {7, true},
        {11, false},
        {13, true},
    }
fmt.Println(s)

[3 5 7]
[{2 true} {3 false} {5 true} {7 true} {11 false} {13 true}]


60 <nil>

#### range

In [12]:
pow := make([]int, 10)
for i := range pow {
    pow[i] = 1 << uint(i) // == 2**i
}
for _, value := range pow {
    fmt.Printf("%d\n", value)
}

1
2
4
8
16
32
64
128
256
512


### closures

In [15]:
func adder(i int) func(int) int {
    sum := i
    return func(x int) int {
        sum += x
        return sum
    }
}

func main() {
    pos, neg := adder(1), adder(2)
    for i := 0; i < 10; i++ {
        fmt.Println(
            pos(i),
            neg(-2*i),
        )
    }
}
main()

1 2
2 0
4 -4
7 -10
11 -18
16 -28
22 -40
29 -54
37 -70
46 -88


### methods

##### methods can take value or a pointer and interpret as a pointer whereas pointer receiver functions need pointer as argument

In [27]:
type Vertex struct {
    X, Y float64
}

func (v Vertex) Abs() float64 {
    return math.Sqrt(v.X*v.X + v.Y*v.Y)
}


v := Vertex{3, 4}
_, _ = fmt.Println(v.Abs())


type MyFloat float64

func (f MyFloat) Abs() float64 {
    if f < 0 {
        return float64(-f)
    }
    return float64(f)
}


f := MyFloat(-math.Sqrt2)
_, _ = fmt.Println(f.Abs())

5
1.4142135623730951


In [29]:
type Vertex struct {
    X, Y float64
}

func (v Vertex) Abs() float64 {
    return math.Sqrt(v.X*v.X + v.Y*v.Y)
}

// pointer receiver
func (v *Vertex) Scale(f float64) {
    v.X = v.X * f
    v.Y = v.Y * f
}


v := Vertex{3, 4}
v.Scale(10)
_, _ = fmt.Println(v.Abs())

50


### interfaces

##### A value of interface type can hold any value that implements those methods.

In [33]:
type Abser interface {
    Abs() float64
}

type MyFloat float64

func (f MyFloat) Abs() float64 {
    if f < 0 {
        return float64(-f)
    }
    return float64(f)
}

type Vertex struct {
    X, Y float64
}

func (v *Vertex) Abs() float64 {
    return math.Sqrt(v.X*v.X + v.Y*v.Y)
}


var a Abser
f := MyFloat(-math.Sqrt2)
v := Vertex{3, 4}

a = f  // a MyFloat implements Abser
a = &v // a *Vertex implements Abser

// In the following line, v is a Vertex (not *Vertex)
// and does NOT implement Abser.
// a = v

_, _ = fmt.Println(a.Abs())

5


#### empty interface

can hold variables of any type

In [35]:
func main() {
    var i interface{}
    describe(i)

    i = 42
    describe(i)

    i = "hello"
    describe(i)
}

func describe(i interface{}) {
    fmt.Printf("(%v, %T)\n", i, i)
}

main()

(<nil>, <nil>)
(42, int)
(hello, string)


#### Stringer : interface with String() method

In [38]:
type Person struct {
    Name string
    Age  int
}

func (p Person) String() string {
    return fmt.Sprintf("%v (%v years)", p.Name, p.Age)
}


a := Person{"Arthur Dent", 42}
z := Person{"Zaphod Beeblebrox", 9001}
_, _ = fmt.Println(a, z)

{Arthur Dent 42} {Zaphod Beeblebrox 9001}
